In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import nltk
import string



In [ ]:
file_path = "/content/drive/MyDrive/NLP /NLP Final Project/plots.csv"

In [ ]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [ ]:
stop_words = set(stopwords.words('english'))
punctuation = set(string.punctuation)
lemmatizer = WordNetLemmatizer()

In [ ]:
def process_text(cell):
    if isinstance(cell, str):
        cell = cell.lower()
        cell = ''.join(char if char not in punctuation else ' ' for char in cell)
        words = cell.split()
        filtered_words = [
            lemmatizer.lemmatize(word) for word in words if word not in stop_words
        ]
        return " ".join(filtered_words)
    return cell

In [ ]:
df = pd.read_csv(file_path)
df = df.applymap(process_text)

<ipython-input-7-cb83073aea89>:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(process_text)


In [ ]:
tf = TfidfVectorizer()
tfidf_matrix = tf.fit_transform(df['Plot Summary'])
cosine_sim_matrix = cosine_similarity(tfidf_matrix, tfidf_matrix)




In [ ]:
import numpy as np
np.random.seed(42)
n = len(df)
feedback_matrix = np.random.randint(-5, 6, size=(n, n))

In [ ]:
def adjust_similarity_with_feedback(sim_matrix, feedback_matrix, boost=0.1, penalty=0.1):
    adjusted_sim_matrix = sim_matrix.copy()
    for i in range(len(feedback_matrix)):
        for j in range(len(feedback_matrix)):
            if feedback_matrix[i, j] > 0:
                adjusted_sim_matrix[i, j] += boost * feedback_matrix[i, j]
            elif feedback_matrix[i, j] < 0:
                adjusted_sim_matrix[i, j] += penalty * feedback_matrix[i, j]
                adjusted_sim_matrix[i, j] = max(0, adjusted_sim_matrix[i, j])
    return adjusted_sim_matrix

In [ ]:
cosine_sim_matrix_adjusted = adjust_similarity_with_feedback(cosine_sim_matrix, feedback_matrix)

cosine_sim_df_adjusted = pd.DataFrame(cosine_sim_matrix_adjusted, index=df['Code'], columns=df['Code'])



In [ ]:
def get_top_5_recommendations(sim_matrix, item_codes):
    top_5_recommendations = {}
    for idx, item in enumerate(item_codes):
        sim_scores = list(enumerate(sim_matrix[idx]))
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
        top_5 = [item_codes[i] for i, score in sim_scores[1:6]]
        top_5_recommendations[item] = top_5

    return top_5_recommendations
item_codes = df['Code'].tolist()


top_5_recs = get_top_5_recommendations(cosine_sim_matrix_adjusted, item_codes)


for item, recommendations in top_5_recs.items():
    print(f"Top 5 recommendations for {item}: {recommendations}")
